In [ ]:
!pip install ipywidgets librosa matplotlib pandas requests torch torchaudio diffusers gtts transformers

In [ ]:
import csv
import re
from typing import List, Dict

import ipywidgets as widgets
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import requests
import torch
import torchaudio
from IPython.display import Audio, display
from diffusers import StableDiffusionPipeline
from gtts import gTTS
from transformers import AutoTokenizer, AutoModelForTextToWaveform
from transformers import pipeline

In [ ]:
# Check if GPU is available and set the device
device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if device == 0 else "CPU")

# **Sentiment Analysis**

In [ ]:
# Load the sentiment analysis pipeline with the appropriate device
sentiment_analyzer = pipeline("sentiment-analysis", device=device)

# Sample texts for sentiment analysis
texts = [
    "I love using AI tools! They make life so much easier.",
    "The service was terrible, and the staff was rude.",
    "This product is okay, not the best but not the worst either.",
    "Absolutely fantastic experience, I highly recommend it!",
    "I'm disappointed with the results, not what I expected.",
]

# Analyze the sentiments
results = sentiment_analyzer(texts)

# Prepare data for visualization
df = pd.DataFrame(results)
df['text'] = texts
df['score'] = df['score'].round(2)

# Map labels to colors for visualization
colors = {'POSITIVE': 'green', 'NEGATIVE': 'red'}

# Plot the results
plt.figure(figsize=(10, 6))
bars = plt.bar(df['text'], df['score'], color=df['label'].map(colors))
plt.xticks(rotation=45, ha="right", fontsize=10)
plt.title("Sentiment Analysis Results", fontsize=16)
plt.ylabel("Confidence Score", fontsize=12)
plt.xlabel("Input Text", fontsize=12)

# Add text labels to each bar
for bar, label, score in zip(bars, df['label'], df['score']):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f"{label} ({score})", ha="center", va="bottom", fontsize=10, color="black")

plt.tight_layout()
plt.show()

# Print tabular results
print("\nDetailed Results:")
print(df)


# **Text Summarization**

In [ ]:
# Load the text summarization pipeline with GPU support
summarizer = pipeline("summarization", device=device)

# Input text
text = """
Climate change is one of the most pressing challenges of the 21st century, with far-reaching consequences for the environment, economies, and societies. Rising global temperatures, caused by increased greenhouse gas emissions from human activities, have led to melting polar ice caps, rising sea levels, and more frequent extreme weather events. These changes threaten biodiversity, disrupt food production, and force millions of people to migrate from vulnerable regions.

Efforts to combat climate change include international agreements such as the Paris Agreement, which aims to limit global warming to below 2 degrees Celsius compared to pre-industrial levels. Countries are committing to reduce their carbon footprints by investing in renewable energy sources, improving energy efficiency, and adopting sustainable practices in agriculture and industry.

However, achieving these goals requires significant collaboration among governments, businesses, and individuals. Technological innovations, such as carbon capture and storage, electric vehicles, and advancements in solar and wind energy, play a crucial role in reducing emissions. Public awareness and education are equally important, as they encourage behavioral changes like reducing waste, conserving energy, and adopting plant-based diets.

Despite these efforts, challenges remain. Many developing countries struggle to balance economic growth with environmental sustainability, and the financial resources needed to implement green technologies are often scarce. Additionally, political resistance and misinformation hinder progress, delaying the urgent actions required to address this global crisis.
"""

# Perform summarization
summary = summarizer(text, max_length=1000, min_length=25, do_sample=False)
summary_text = summary[0]['summary_text']

# Display the summary
print("\nOriginal Text:")
print(text)
print("\nGenerated Summary:")
print(summary_text)

# **Named Entity Recognition**

In [ ]:
# Load the NER pipeline with GPU support
model_name = "dslim/bert-base-NER"  # Example model
ner = pipeline("ner", grouped_entities=True, model=model_name, device=device)


# Preprocessing: Clean and normalize input text
def preprocess_text(text: str) -> List[str]:
    """
    Clean and normalize the input text for better NER performance.
    - Normalize whitespace and punctuation.
    - Split text into sentences for more structured processing.
    """
    text = re.sub(r"\s+", " ", text.strip())  # Normalize whitespace
    text = re.sub(r"[^\w\s.,!?-]", "", text)  # Remove special characters
    sentences = re.split(r"(?<=[.!?])\s+", text)  # Split into sentences
    return sentences


# Postprocessing: Merge and refine entities
def postprocess_entities(entities: List[Dict]) -> List[Dict]:
    """
    Postprocess NER results to:
    - Merge fragmented entities (e.g., "A ##zure" -> "Azure").
    - Remove low-confidence predictions (score < 0.85).
    - Handle overlapping entities.
    """
    merged_entities = []
    temp_entity = None

    for entity in entities:
        if entity["word"].startswith("##"):  # Merge subwords
            if temp_entity:
                temp_entity["word"] += entity["word"].replace("##", "")
                temp_entity["score"] = (temp_entity["score"] + entity["score"]) / 2
        else:
            if temp_entity:  # Append the previous entity
                merged_entities.append(temp_entity)
            temp_entity = entity

    if temp_entity:  # Append the last entity
        merged_entities.append(temp_entity)

    # Filter low-confidence entities
    merged_entities = [e for e in merged_entities if e["score"] >= 0.85]

    # Remove duplicates
    seen = set()
    unique_entities = []
    for entity in merged_entities:
        if entity["word"] not in seen:
            unique_entities.append(entity)
            seen.add(entity["word"])

    return unique_entities


# Save entities to a CSV file
def save_entities_to_csv(entities: List[Dict], filename: str = "entities.csv") -> None:
    """
    Save the detected entities to a CSV file.
    """
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Entity", "Label", "Score"])
        for entity in entities:
            writer.writerow([entity["word"], entity["entity_group"], f"{entity['score']:.2f}"])
    print(f"\nEntities saved to {filename}")


# Input text
text = """
Dear Team,

I hope this email finds you well. I’m excited to share some key updates regarding our ongoing initiatives and upcoming projects.

Partnership Announcement:
Last Friday, on January 19, 2025, we finalized a strategic partnership with OpenAI to integrate Azure OpenAI Service into our North America operations. This collaboration is expected to enhance our AI-driven analytics platform, InsightX, by Q2.

Office Expansion:
As part of our global expansion strategy, we’ll be opening new offices in Berlin and Tokyo by June 2025. Both locations were chosen based on their proximity to key clients such as Siemens and SoftBank.

Quarterly Revenue Milestone:
I’m thrilled to report that our Q1 revenue has exceeded $50 million, driven by strong adoption of our flagship product, DataVista.

Team Summit:
Please mark your calendars for the Annual Leadership Summit, which will be held at the Grand Hyatt in San Francisco from April 3–5, 2025. This year’s keynote speaker will be Dr. Maria Rodriguez, a leading expert in machine learning and data ethics.

Action Required:

Regional Managers: Submit your budget reports for Q2 by February 15, 2025.
Product Teams: Prepare a demo of your progress with InsightX for the next review meeting scheduled on March 10, 2025.
Please let me know if you have any questions or require additional resources.

Best regards,
Alex Carter
Chief Operating Officer
VisionTech Solutions
"""

# Preprocess the text
sentences = preprocess_text(text)

# Perform named entity recognition
all_entities = []
for sentence in sentences:
    raw_entities = ner(sentence)
    all_entities.extend(raw_entities)

# Postprocess the entities
processed_entities = postprocess_entities(all_entities)

# ANSI colors for different entity types
entity_colors = {
    "PER": "\033[94m",  # Blue for Person
    "LOC": "\033[92m",  # Green for Location
    "ORG": "\033[91m",  # Red for Organization
    "MISC": "\033[93m",  # Yellow for Miscellaneous
    "RESET": "\033[0m"  # Reset to default
}

# Highlight entities in the text
highlighted_text = ""

for sentence in sentences:
    # Keep track of where entities start and stop
    sentence_entities = [
        e for e in processed_entities if e['word'] in sentence
    ]

    # Sort entities by their starting index in the sentence for consistent processing
    sentence_entities.sort(key=lambda e: sentence.find(e['word']))

    current_index = 0
    for entity in sentence_entities:
        # Find the entity's position in the sentence
        start_index = sentence.find(entity['word'], current_index)
        end_index = start_index + len(entity['word'])

        # Append the non-entity text before the current entity
        highlighted_text += sentence[current_index:start_index]

        # Highlight the entity text
        color = entity_colors.get(entity['entity_group'], entity_colors["RESET"])
        highlighted_text += f"{color}{sentence[start_index:end_index]}{entity_colors['RESET']}"

        # Move the current index to after the entity
        current_index = end_index

    # Append any remaining text after the last entity
    highlighted_text += sentence[current_index:] + "\n"

# Display the highlighted text in the log
print("\nNamed Entity Recognition Result (Highlighted):\n")
print(highlighted_text)

# Display detected entities in tabular format
print("\nEntities Detected:")
print(f"{'Entity':<20}{'Label':<10}{'Score':<6}")
print("-" * 40)
for entity in processed_entities:
    print(f"{entity['word']:<20}{entity['entity_group']:<10}{entity['score']:.2f}")

# Save entities to a CSV file
save_entities_to_csv(processed_entities)

# **Text-to-speech**

In [ ]:
# Function to generate and play speech
def text_to_speech(text, language="en", output_file="speech.mp3"):
    """
    Convert text to speech using gTTS and play the audio.

    Args:
    - text (str): Text to convert to speech.
    - language (str): Language code (default: "en" for English).
    - output_file (str): Name of the output audio file (default: "speech.mp3").
    """
    try:
        # Generate speech
        print("Generating speech...")
        tts = gTTS(text, lang=language)
        tts.save(output_file)
        print(f"Speech saved to {output_file}")

        # Display the generated audio
        print("Playing generated speech...")
        display(Audio(output_file))

    except ValueError as e:
        print(f"Error: {e}")
        print("Please check the language code or the input text.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# User inputs
text = input("Enter the text to convert to speech: ")
language = input("Enter the language code (e.g., 'en' for English, 'es' for Spanish): ").strip()

# Validate inputs
if not text.strip():
    print("Error: Text input is empty. Please enter valid text.")
elif len(language) != 2:
    print("Error: Invalid language code. Please enter a valid 2-letter language code (e.g., 'en').")
else:
    # Generate and play speech
    text_to_speech(text, language)


# Speech-to-text

In [ ]:
# Load the Whisper model for speech-to-text with GPU support
stt_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

# Default audio file (downloaded if not uploaded by the user)
default_audio_url = "https://www2.cs.uic.edu/~i101/SoundFiles/BabyElephantWalk60.wav"
default_audio_path = "default_audio.wav"

# Download the default audio file
response = requests.get(default_audio_url)
with open(default_audio_path, "wb") as f:
    f.write(response.content)
print("\nDefault audio file downloaded.")

# File upload widget
uploader = widgets.FileUpload(accept='.mp3,.wav', multiple=False)


# Function to transcribe audio
def transcribe_audio(file_path):
    # Load and display the audio waveform
    audio, sr = librosa.load(file_path, sr=None)
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(audio, sr=sr, alpha=0.8)
    plt.title("Audio Waveform", fontsize=16)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()

    # Transcribe the audio
    print("\nTranscribing audio...")
    transcription = stt_pipeline(file_path)

    # Display the transcription
    print("\nTranscription:")
    print(transcription['text'])

    # Save transcription to a text file
    transcription_file = "transcription.txt"
    with open(transcription_file, "w") as f:
        f.write(transcription['text'])
    print(f"\nTranscription saved to '{transcription_file}'.")

    # Play the audio
    display(Audio(file_path, autoplay=True))


# Function to handle file uploads
def handle_upload(change):
    if uploader.value:
        for filename, file_info in uploader.value.items():
            file_path = filename
            # Save the uploaded file locally
            with open(file_path, "wb") as f:
                f.write(file_info["content"])
            # Transcribe the uploaded file
            transcribe_audio(file_path)
    else:
        print("No file uploaded.")


# Connect the uploader to the function
uploader.observe(handle_upload, names="value")

# Display the upload widget and process default audio if no upload is done
print("\nUpload an audio file or use the default audio file:")
display(uploader)

# Transcribe the default audio file if no upload is performed
transcribe_audio(default_audio_path)


# **Text Generation**

In [ ]:
# Load the text generation pipeline with GPU support
generator = pipeline("text-generation", model="gpt2", device=device)

# Input prompt and settings
prompt = "The future of AI in healthcare"
max_length = 50  # Maximum length of generated text
num_return_sequences = 3  # Number of outputs to generate

# Generate text
print("\nGenerating text...")
generated_texts = generator(prompt, max_length=max_length, num_return_sequences=num_return_sequences)

# Display generated texts
print("\nGenerated Texts:")
for i, gen in enumerate(generated_texts):
    print(f"\nOutput {i + 1}:")
    print(gen['generated_text'])


# **Text-to-image**

In [ ]:
# Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)

# Input prompt and image settings
prompt = "A futuristic cityscape with flying cars, highly detailed, photorealistic"
height = 512
width = 512

# Generate the image
print("\nGenerating the image...")
try:
    image = pipe(prompt, height=height, width=width).images[0]

    # Display the image with metadata
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Generated Image for Prompt:\n'{prompt}'", fontsize=16)
    plt.show()

    # Save the image
    image_path = "generated_image.png"
    image.save(image_path)
    print(f"Image saved to {image_path}.")

except Exception as e:
    print("Error during image generation:", e)


# **Text-to-music**

In [ ]:
# Step 3: Load the MusicGen model directly
print("Loading the MusicGen model...")
tokenizer = AutoTokenizer.from_pretrained("facebook/musicgen-large")
model = AutoModelForTextToWaveform.from_pretrained("facebook/musicgen-large")


# Step 4: Define the music generation function
def generate_music(prompt, duration=10, save_path="generated_music.wav"):
    """
    Generate music based on a text prompt using MusicGen.

    Args:
    - prompt (str): The text description of the music.
    - duration (int): Approximate duration of the music in seconds.
    - save_path (str): Path to save the generated audio (default: "generated_music.wav").

    Returns:
    - str: Path to the saved audio file.
    """
    print("Generating music...")

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate the audio waveform
    generated_waveform = model.generate(**inputs, max_new_tokens=duration * 50)  # Approx. 50 tokens/sec

    # Convert the waveform to an audio file
    torchaudio.save(save_path, generated_waveform.audio, sample_rate=generated_waveform.sample_rate)

    print(f"Music generated and saved to {save_path}")
    return save_path


# Step 5: Generate music with a custom duration
prompt = "A calm and relaxing piano melody for a peaceful evening"
duration = 20  # Duration in seconds

output_file = generate_music(prompt, duration)

# Step 6: Play the generated music
print("Playing the generated music...")
display(Audio(output_file))


Support for third party widgets will remain active for the duration of the session. To disable support:

Support for third party widgets will remain active for the duration of the session. To disable support: